# Sentiment Analysis Demo

Notebook from [Eric Elmoznino](https://github.com/EricElmoznino/lighthouse_nlp_II).

![workflow](images/sentiment_workflow.jpg)

# Loading the dataset

Dataset is downloaded from [here](https://www.kaggle.com/code/kerneler/starter-imdb-master-csv-19b6829a-2/data).

In [1]:
import pandas as pd
imdb_df = pd.read_csv('data/imdb_master.csv', encoding = "ISO-8859-1")
imdb_df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [2]:
imdb_df['label'].value_counts()

unsup    50000
neg      25000
pos      25000
Name: label, dtype: int64

In [3]:
# Only consider positive and negative reviews
imdb_df = imdb_df[imdb_df['label'].str.startswith(('pos','neg'))]

# Preprocessing to a numeric representation

The current data is in the form of moview reviews (text paragraphs) and their targets (`pos` or `neg`). 
We need to encode movie reviews into feature vectors so that we can train supervised machine learning models with `scikit-learn`. 
How can we do this?

#### Create binarized word frequency counts (`X_binary`)
Turn the text into sparse vector of word frequency counts using [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from  `scikit-learn`. 

When you reproduce this, explore the arguments of `CountVectorizer` (e.g., [`stop_words`](https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words), `ngram_range`, `max_features`, `min_df`, `tokenizer`, and `binary`).  

The intuition behind using binarized representation is that for sentiment analysis word occurrence may matter more than word frequency. For instance, the occurrence of the word _excellent_ tells us a lot and the fact that it occurs four times may not tell us much more. This is just a hypothesis that you could test, however.

In [4]:
# For tokenization
import nltk
# For converting words into frequency counts
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# First step in pipeline
# Keep words that appear in atleast 2 documents, keeps 5000 most common words
preprocessor = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=5000, binary=True)

In [6]:
# Demo the preprocessor:
X_example = preprocessor.fit_transform(imdb_df['review'].iloc[:1000])
print(f'Preprocessing output shape: {X_example.shape}')

# Show the process for the first datapoint
first_datapoint = imdb_df['review'].iloc[0]
print(f'First datapoint: {first_datapoint[:100]}')

first_tokens = nltk.word_tokenize(first_datapoint)
print(f'First datapoint tokens: {first_tokens[:10]}')

first_bow = preprocessor.transform([first_datapoint])
first_bow.maxprint = 5  # Change how many of the non-zero elements are printing to not clutter the notebook
print(f'First datapoint Binary Bag of Words (sparse) representation:\n{first_bow}')
print(f'First datapoint Binary Bag of Words (dense) representation:\n{first_bow.todense()}')

Preprocessing output shape: (1000, 5000)
First datapoint: Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrifi
First datapoint tokens: ['Once', 'again', 'Mr.', 'Costner', 'has', 'dragged', 'out', 'a', 'movie', 'for']
First datapoint Binary Bag of Words (sparse) representation:
  (0, 13)	1
  (0, 20)	1
  :	:
  (0, 4866)	1
  (0, 4889)	1
  (0, 4898)	1
First datapoint Binary Bag of Words (dense) representation:
[[0 0 0 ... 0 0 0]]


### Question
What is the disadvantage of using something like a *Bag of Words* representation for the documents in sentiment analysis?

### Answer
You lose the word order, and therefore the syntax. For instance, negation is lost.

# Modeling

In [7]:
from sklearn.naive_bayes import BernoulliNB # Bernoulli because we have binary features
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('preprocessing', preprocessor), 
                     ('model', BernoulliNB())])

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(imdb_df['review'], imdb_df['label'], test_size=0.20, random_state=27)

In [9]:
pipeline.fit(X_train, y_train)
train_accuracy = pipeline.score(X_train, y_train)
test_accuracy = pipeline.score(X_test, y_test)

print(f'Train accuracy:\t{train_accuracy}')
print(f'Test accuracy:\t{test_accuracy}')

Train accuracy:	0.85295
Test accuracy:	0.8397
